# Data Preparation

## Setup

In [3]:
# Import useful libraries
import numpy as np
import pandas as pd
from scipy import stats
import os
import re
import pickle
import sys
import time
import psycopg2

In [ ]:
# # Setup Repository
# with open("repo_info.txt", "r") as repo_info:
#     path_to_repo = repo_info.readline()

# path_to_data = f"{path_to_repo}data/"
# path_to_raw = f"{path_to_data}raw/"
# path_to_processed = f"{path_to_data}processed/"

## Import Data

In [4]:
path_to_raw="raw_data"
df_raw = pd.read_csv(f'{path_to_raw}/all_data_raw.csv.gzip', compression = 'gzip')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625220 entries, 0 to 625219
Data columns (total 63 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   hadm_id                     625220 non-null  int64  
 1   subject_id                  625220 non-null  int64  
 2   admittime                   625220 non-null  object 
 3   dischtime                   625220 non-null  object 
 4   deathtime                   141883 non-null  object 
 5   race                        625220 non-null  object 
 6   admission_type              625220 non-null  object 
 7   admission_location          625220 non-null  object 
 8   insurance                   625220 non-null  object 
 9   language                    625220 non-null  object 
 10  marital_status              578293 non-null  object 
 11  discharge_location          624494 non-null  object 
 12  first_careunit              625220 non-null  object 
 13  last_careunit 

In [22]:
#Then we drop any duplicates
# df = df_raw.drop_duplicates(subset=['hadm_id']).reset_index()
# df = df_raw.drop_duplicates()

In [5]:
#Because there are some subtle differences between the records belonging to the same hadm id, we can do some process
#For float column difference, we extract the average value
#For others, we keep the mode value
def aggregate_mode(x):
    modes = stats.mode(x)[0]
    return modes[0] if len(modes) else np.nan

def custom_agg(series):
    if series.dtype == float:
        return series.mean()
    else:
        return aggregate_mode(series)

diff_cols_per_group = df_raw.groupby('hadm_id').agg(lambda x: x.nunique(dropna=False) > 1)
cols_to_aggregate = diff_cols_per_group.any()
df = df_raw.groupby('hadm_id').agg({col: custom_agg for col in df_raw.columns if col in cols_to_aggregate and col != 'hadm_id'})

df.head()


C:\Users\14297\AppData\Local\Temp\ipykernel_21296\3656087538.py:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  modes = stats.mode(x)[0]
C:\Users\14297\AppData\Local\Temp\ipykernel_21296\3656087538.py:5: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  modes = stats.mode(x)[0]


,subject_id,admittime,dischtime,deathtime,race,admission_type,admission_location,insurance,language,marital_status,...,icd_code,proc_icd,diag_icd,age_cat,type_stay,prev_adm,dest_discharge,emergency_dpt,icd_chapter,origin_patient
hadm_id,,,,,,,,,,,,,,,,,,,,,
20000094,14046553,2150-03-02 00:00:00,2150-03-03 09:21:00,2150-03-03 09:21:00,WHITE,URGENT,TRANSFER FROM HOSPITAL,Other,ENGLISH,WIDOWED,...,I509,0,"I509 ,K7200 ,R570 ,N179 ,I2510 ,Z9861 ...",4,1-Medical,1-No hospitalization,2-Other,Yes,29,2-Other
20000147,14990224,2121-08-30 16:33:00,2121-09-03 15:45:00,NaN,WHITE - OTHER EUROPEAN,URGENT,TRANSFER FROM HOSPITAL,Medicare,ENGLISH,MARRIED,...,I213,"02100Z9,B211YZZ,021209W,06BQ4ZZ,5A1221Z","I213 ,I5021 ,E1121 ,E1165 ,I25119 ,I110 ...",4,3-Surgical,1-No hospitalization,2-Other,Yes,29,2-Other
20000808,16788749,2180-01-12 20:59:00,2180-01-25 12:10:00,NaN,WHITE,EW EMER.,EMERGENCY ROOM,Other,ENGLISH,SINGLE,...,2252,"0151,0239,0239,0212","2252 ,3485 ,3313 ,311 ,53081 ,4019 ...",3,3-Surgical,1-No hospitalization,1-Home,No,2,2-Other
20001305,16003661,2178-03-25 02:58:00,2178-03-27 19:23:00,2178-03-27 19:23:00,WHITE,OBSERVATION ADMIT,EMERGENCY ROOM,Medicare,ENGLISH,SINGLE,...,J9601,"5A1945Z,0BH17EZ,02HV33Z","J9601 ,I214 ,R578 ,N170 ,R64 ,D684 ...",4,1-Medical,1-No hospitalization,2-Other,No,30,2-Other
20001361,14577567,2143-05-04 14:55:00,2143-05-18 16:58:00,NaN,WHITE,EW EMER.,EMERGENCY ROOM,Medicaid,ENGLISH,SINGLE,...,3481,"9604,9671,3893,3891,0331,966","3481 ,5845 ,51881 ,72888 ,34839 ,30401 ...",2,1-Medical,1-No hospitalization,2-Other,No,6,2-Other


In [6]:
# And print info our dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48924 entries, 20000094 to 29999828
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   subject_id                  48924 non-null  int64  
 1   admittime                   48924 non-null  object 
 2   dischtime                   48924 non-null  object 
 3   deathtime                   5885 non-null   object 
 4   race                        48924 non-null  object 
 5   admission_type              48924 non-null  object 
 6   admission_location          48924 non-null  object 
 7   insurance                   48924 non-null  object 
 8   language                    48924 non-null  object 
 9   marital_status              45077 non-null  object 
 10  discharge_location          48732 non-null  object 
 11  first_careunit              48924 non-null  object 
 12  last_careunit               48924 non-null  object 
 13  icu_los              

In [7]:
# Convert the deathtime column to a datetime-column
df['deathtime'] = pd.to_datetime(df.deathtime, errors = 'coerce')
# Remove any observation with a recorded death case
df_clean = df[np.isnat(df['deathtime'])].copy()
print(f"Number of observations with a deathtime: {df.shape[0] - df_clean.shape[0]}")

Number of observations with a deathtime: 5885


In [8]:
# We then remove all patients below 18 years old
df_clean = df_clean.loc[df_clean.age >= 18]
print(f"N. of observations: {df_clean.shape[0]}")

N. of observations: 43039


In [9]:
df_clean.to_csv("df_clean.csv")

# Merge notes data into the dataset

## Process notes to get discharge notes

In [37]:
# #The following is based on personal psql setting
# host = 'localhost'  
# port = '5432'  
# sqluser = 'postgres'  
# password = 'mydb'  
# dbname = 'mimiciv' 
# schema_name1 = 'mimiciv_hosp' 
# schema_name2 = 'mimiciv_icu'
# schema_name3 = 'mimiciv_derived' 
# schema_name4 = 'mimiciv_note'
# schema_search_path = f"{schema_name1}, {schema_name2}, {schema_name3}, {schema_name4}"


# try:
#     con = psycopg2.connect(
#         host=host,
#         port=port,
#         database=dbname,
#         user=sqluser,
#         password=password
#     )

#     # due to the difference in data structure, notes in mimiciv is a table that we create(materialized view)

#     query = \
#     """
#     select * from note_filter
#     """
#     # Connect to local postgres version of mimic
#     con = psycopg2.connect(host='localhost', database=dbname, user=sqluser, password=password)
#     cur = con.cursor()

#     # Perform SQL query
#     cur.execute('SET search_path to ' + schema_search_path)
#     text = pd.read_sql_query(query,con)

# finally:
#     if con is not None:
#         con.close()

C:\Users\14297\AppData\Local\Temp\ipykernel_8064\2552751421.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  text = pd.read_sql_query(query,con)


In [9]:
text_d=pd.read_csv("../datasets/discharge_note.csv")
text_r=pd.read_csv("../datasets/radiology_note.csv")

In [12]:
text_d.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,first_careunit,last_careunit,age,gender,marital_status,insurance,discharge_text
0,10004749,28691602,2127-04-04 20:01:00,2127-04-07 17:56:00,NaN,NaN,NaN,32,F,SINGLE,Other,\nName: ___ Unit No: ___\n ...
1,10005001,25115899,2164-10-12 00:00:00,2164-10-14 17:24:00,NaN,NaN,NaN,40,F,SINGLE,Other,\nName: ___ Unit No: ___\n \...
2,10005123,20470681,2129-08-17 17:45:00,2129-08-19 16:04:00,NaN,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),54,M,MARRIED,Other,\nName: ___ Unit No: ___\...
3,10005565,29140012,2180-08-21 23:01:00,2180-08-22 18:50:00,NaN,NaN,NaN,61,F,MARRIED,Other,\nName: ___ Unit No: ...
4,10005605,24283979,2120-05-20 23:48:00,2120-05-23 14:30:00,NaN,NaN,NaN,62,F,SINGLE,Other,\nName: ___ Unit No: __...


In [16]:
# text=pd.read_csv("../dataset/note_filter.csv", index=False)
text = pd.merge(text_d, text_r, 
                       on="hadm_id", 
                       how="outer", 
                       suffixes=('_dn', '_rn'))

In [17]:
#Fillna in discharge with radiology records
for column in text.columns:
    if column not in ['discharge_text', 'radiology_text', 'hadm_id']:
        if '_dn' in column:
            corresponding_radiology_column = column.replace('_dn', '_rn')
            text[column] = text[column].fillna(text[corresponding_radiology_column])
columns_to_drop = [column for column in text.columns if '_rn' in column]
text.drop(columns=columns_to_drop, inplace=True)

text.columns = [column.replace('_dn', '') for column in text.columns]




In [19]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1736566 entries, 0 to 1736565
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   subject_id      float64
 1   hadm_id         int64  
 2   admittime       object 
 3   dischtime       object 
 4   deathtime       object 
 5   first_careunit  object 
 6   last_careunit   object 
 7   age             float64
 8   gender          object 
 9   marital_status  object 
 10  insurance       object 
 11  discharge_text  object 
 12  radiology_text  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 185.5+ MB


In [20]:
text['subject_id'] = text['subject_id'].astype('Int64')
text['age'] = text['age'].astype('Int64')
text['admittime'] = pd.to_datetime(text['admittime'])
text['dischtime'] = pd.to_datetime(text['dischtime'])
text['deathtime'] = pd.to_datetime(text['deathtime'])


In [21]:
text.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,first_careunit,last_careunit,age,gender,marital_status,insurance,discharge_text,radiology_text
0,10004749,28691602,2127-04-04 20:01:00,2127-04-07 17:56:00,NaT,NaN,NaN,32,F,SINGLE,Other,\nName: ___ Unit No: ___\n ...,INDICATION: Severe abdominal and pelvic pain....
1,10004749,28691602,2127-04-04 20:01:00,2127-04-07 17:56:00,NaT,NaN,NaN,32,F,SINGLE,Other,\nName: ___ Unit No: ___\n ...,INDICATION: Pelvic pain.\n\nCOMPARISONS: CT ...
2,10005001,25115899,2164-10-12 00:00:00,2164-10-14 17:24:00,NaT,NaN,NaN,40,F,SINGLE,Other,\nName: ___ Unit No: ___\n \...,NaN
3,10005123,20470681,2129-08-17 17:45:00,2129-08-19 16:04:00,NaT,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),54,M,MARRIED,Other,\nName: ___ Unit No: ___\...,INDICATION: ___ year old man with congenital ...
4,10005123,20470681,2129-08-17 17:45:00,2129-08-19 16:04:00,NaT,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),54,M,MARRIED,Other,\nName: ___ Unit No: ___\...,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...


In [22]:
# Remove any duplicates
text.drop_duplicates(inplace = True)

In [25]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1263674 entries, 0 to 1736565
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   subject_id      1263674 non-null  Int64         
 1   hadm_id         1263674 non-null  int64         
 2   admittime       1263674 non-null  datetime64[ns]
 3   dischtime       1263674 non-null  datetime64[ns]
 4   deathtime       104684 non-null   datetime64[ns]
 5   first_careunit  568516 non-null   object        
 6   last_careunit   568516 non-null   object        
 7   age             1263674 non-null  Int64         
 8   gender          1263674 non-null  object        
 9   marital_status  1204778 non-null  object        
 10  insurance       1263674 non-null  object        
 11  discharge_text  1187654 non-null  object        
 12  radiology_text  1199050 non-null  object        
dtypes: Int64(2), datetime64[ns](3), int64(1), object(7)
memory usage: 137.4+

In [23]:
text.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,first_careunit,last_careunit,age,gender,marital_status,insurance,discharge_text,radiology_text
0,10004749,28691602,2127-04-04 20:01:00,2127-04-07 17:56:00,NaT,NaN,NaN,32,F,SINGLE,Other,\nName: ___ Unit No: ___\n ...,INDICATION: Severe abdominal and pelvic pain....
1,10004749,28691602,2127-04-04 20:01:00,2127-04-07 17:56:00,NaT,NaN,NaN,32,F,SINGLE,Other,\nName: ___ Unit No: ___\n ...,INDICATION: Pelvic pain.\n\nCOMPARISONS: CT ...
2,10005001,25115899,2164-10-12 00:00:00,2164-10-14 17:24:00,NaT,NaN,NaN,40,F,SINGLE,Other,\nName: ___ Unit No: ___\n \...,NaN
3,10005123,20470681,2129-08-17 17:45:00,2129-08-19 16:04:00,NaT,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),54,M,MARRIED,Other,\nName: ___ Unit No: ___\...,INDICATION: ___ year old man with congenital ...
4,10005123,20470681,2129-08-17 17:45:00,2129-08-19 16:04:00,NaT,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),54,M,MARRIED,Other,\nName: ___ Unit No: ___\...,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...


Our discharge notes are not unique for the combination subject_id and hadm_id. We will merge all the notes.

In [26]:
print(f"Number of duplicate text: {round(text.duplicated(subset = ['subject_id', 'hadm_id']).sum()/text.shape[0]*100, 2)}%")

Number of duplicate text: 70.38%


In [30]:
text['discharge_text'] = text.groupby(['subject_id', 'hadm_id', 'admittime']).discharge_text.transform(lambda x: '\n'.join(x.astype(str)))
text['radiology_text'] = text.groupby(['subject_id', 'hadm_id', 'admittime']).radiology_text.transform(lambda x: '\n'.join(x.astype(str)))

In [ ]:
text_clean = text[['hadm_id', 'subject_id', 'discharge_text','radiology_text']].drop_duplicates()
print(f"Duplicates removed: {text.shape[0] - text_clean.shape[0]}")

## Merge together

In [ ]:
df_full = pd.merge(df_clean, text_clean, on = ['hadm_id', 'subject_id'], indicator = True, how = 'left', validate = '1:1')
print(f"Check merges:\n{df_full._merge.value_counts()}")

In [ ]:
# Now remove the failed merges
df_full = df_full.loc[df_full._merge == 'both']
df_full.drop(columns = ['_merge'], inplace = True)

In [ ]:
# Finally save the file to a CSV
path_to_processed = "processed_data"
df_full.to_csv(f'{path_to_processed}/df_mixed.csv.gzip', compression = 'gzip', index = False)